<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Building a Real-time Recommendation API

This reference architecture shows the full lifecycle of building a recommendation system. It walks through the creation of appropriate azure resources, training a recommendation model using a Virtual Machine or Databricks, and deploying it as an API. It uses Azure Cosmos DB, Azure Machine Learning, and Azure Kubernetes Service. 

This architecture can be generalized for many recommendation engine scenarios, including recommendations for products, movies, and news. 
### Architecture
![architecture](https://camo.githubusercontent.com/35ffc0c1bb61f8cc928f02589941376d3005c82d/68747470733a2f2f7265636f64617461736574732e626c6f622e636f72652e77696e646f77732e6e65742f696d616765732f7265636f2d617263682e706e67 "Architecture")

**Scenario**: A media organization wants to provide movie or video recommendations to its users. By providing personalized recommendations, the organization meets several business goals, including increased click-through rates, increased engagement on site, and higher user satisfaction.

In this reference, we train and deploy a real-time recommender service API that can provide the top 10 movie recommendations for a given user. 

### Components
This architecture consists of the following key components:
* [Azure Databricks](https://docs.microsoft.com/en-us/azure/azure-databricks/what-is-azure-databricks)<sup>1)</sup> is used as a development environment to prepare input data and train the recommender model on a Spark cluster. Azure Databricks also provides an interactive workspace to run and collaborate on notebooks for any data processing or machine learning tasks.
* [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes)(AKS) is used to deploy and operationalize a machine learning model service API on a Kubernetes cluster. AKS hosts the containerized model, providing scalability that meets throughput requirements, identity and access management, and logging and health monitoring. 
* [Azure Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/introduction) is a globally distributed database service used to store the top 10 recommended movies for each user. Azure Cosmos DB is ideal for this scenario as it provides low latency (10 ms at 99th percentile) to read the top recommended items for a given user. 
* [Azure Machine Learning Service](https://docs.microsoft.com/en-us/azure/machine-learning/service/) is a service used to track and manage machine learning models, and then package and deploy these models to a scalable Azure Kubernetes Service environment.

<sup>1) Here, we are just giving an example of using Azure Databricks. Any platforms listed in [SETUP](https://github.com/microsoft/recommenders/blob/master/SETUP.md) can be used as well.</sup>


### Table of Contents.
0. [File Imports](#0-File-Imports)
1. [Service Creation](#1-Service-Creation)
2. [Training and evaluation](#2-Training)
3. [Operationalization](#3.-Operationalize-the-Recommender-Service)

## Setup
To run this notebook on Azure Databricks, you should setup Azure Databricks by following the appropriate sections in the repository [SETUP instructions](https://github.com/microsoft/recommenders/blob/master/SETUP.md) and import this notebook into your Azure Databricks Workspace (see instructions [here](https://docs.azuredatabricks.net/user-guide/notebooks/notebook-manage.html#import-a-notebook)).

Please note: This notebook **REQUIRES** that you add the dependencies to support **operationalization**. See [SETUP](https://github.com/microsoft/recommenders/blob/master/SETUP.md) for details.


## 0 File Imports

In [1]:
import os
import sys
sys.path.append("../../")
import time
import urllib

from azure.common.client_factory import get_client_from_cli_profile
import azure.mgmt.cosmosdb
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice
from azureml.exceptions import WebserviceException
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.environment import SparkPackage
import pydocumentdb.document_client as document_client
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType

from reco_utils.common.timer import Timer
from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.dataset import movielens
from reco_utils.dataset.cosmos_cli import find_collection, read_collection, read_database, find_database
from reco_utils.dataset.download_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from reco_utils.common.notebook_utils import is_databricks

print("Azure SDK version:", azureml.core.VERSION)

Azure SDK version: 1.0.69


In JupyterHub, environment variables defined in `.../etc/conda/activate.d` may not be activated. If so, run the following cell to set PySpark environment variables. Make sure your conda environment path is `/anaconda/envs/reco_pyspark` or change the paths in the script.

In [2]:
os.environ["PYSPARK_PYTHON"]="/anaconda/envs/reco_pyspark/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/anaconda/envs/reco_pyspark/bin/python"

In [3]:
# Start spark session if needed
if not is_databricks():
    cosmos_connector = (
        "https://search.maven.org/remotecontent?filepath=com/microsoft/azure/"
        "azure-cosmosdb-spark_2.3.0_2.11/1.3.3/azure-cosmosdb-spark_2.3.0_2.11-1.3.3-uber.jar"
    )
    jar_filepath = maybe_download(url=cosmos_connector, filename="cosmos.jar")
    spark = start_or_get_spark("ALS", memory="10g", jars=[jar_filepath])
    sc = spark.sparkContext
display(sc)

<SparkContext master=local[*] appName=ALS>

## 1 Service Creation
Modify the **Subscription ID** to the subscription you would like to deploy to and set the resource name variables.

#### Services created by this notebook:
1. [Azure ML Service](https://azure.microsoft.com/en-us/services/machine-learning-service/)
1. [Azure Cosmos DB](https://azure.microsoft.com/en-us/services/cosmos-db/)
1. [Azure Container Registery](https://docs.microsoft.com/en-us/azure/container-registry/)
1. [Azure Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/)
1. [Azure Application Insights](https://azure.microsoft.com/en-us/services/monitor/)
1. [Azure Storage](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-overview)
1. [Azure Key Vault](https://azure.microsoft.com/en-us/services/key-vault/)
1. [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/)

**Add your Azure subscription ID**

In [4]:
# Add your subscription ID
subscription_id = ""

# Set your workspace name
workspace_name = "o16n-test"
resource_group = "{}-rg".format(workspace_name)

# Set your region to deploy Azure ML workspace
location = "eastus"

# AzureML service and Azure Kubernetes Service prefix
service_name = "mvl-als"

In [ ]:
# Login for Azure CLI so that AzureML can use Azure CLI login credentials
!az login

In [ ]:
# Change subscription if needed
!az account set --subscription {subscription_id}

In [ ]:
# Check account
!az account show

In [5]:
# CosmosDB
account_name = "{}-ds-sql".format(workspace_name)
# account_name for CosmosDB cannot have "_" and needs to be less than 31 chars
account_name = account_name.replace("_", "-")[:31]
cosmos_database = "recommendations"
cosmos_collection = "user_recommendations_als"

# AzureML resource names
model_name = "{}-reco.mml".format(service_name)
aks_name = "{}-aks".format(service_name)
container_image_name = "{}-img".format(service_name)

In [6]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [7]:
userCol = "UserId"
itemCol = "MovieId"
ratingCol = "Rating"

train_data_path = "train"
test_data_path = "test"

### 1.1 Import or create the AzureML Workspace. 
This command will check if the AzureML Workspace exists or not, and will create the workspace if it doesn't exist.

In [8]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group, 
    location=location,
    exist_ok=True
)

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


### 1.2 Create a Cosmos DB to store recommendation results

This step will take some time to create CosmosDB resources.

In [9]:
# explicitly pass subscription_id in case user has multiple subscriptions
client = get_client_from_cli_profile(
    azure.mgmt.cosmosdb.CosmosDB,
    subscription_id=subscription_id
)

async_cosmosdb_create = client.database_accounts.create_or_update(
    resource_group,
    account_name,
    {
        'location': location,
        'locations': [{
            'location_name': location
        }]
    }
)
account = async_cosmosdb_create.result()

my_keys = client.database_accounts.list_keys(resource_group, account_name)
master_key = my_keys.primary_master_key
endpoint = "https://" + account_name + ".documents.azure.com:443/"

# DB client
client = document_client.DocumentClient(endpoint, {'masterKey': master_key})

if not find_database(client, cosmos_database):
    db = client.CreateDatabase({'id': cosmos_database })
    print("Database created")
else:
    db = read_database(client, cosmos_database)
    print("Database found")

# Create collection options
options = dict(offerThroughput=11000)

# Create a collection
collection_definition = {
    'id': cosmos_collection,
    'partitionKey': {'paths': ['/id'],'kind': 'Hash'}
}
if not find_collection(client, cosmos_database, cosmos_collection):
    collection = client.CreateCollection(
        db['_self'], 
        collection_definition,
        options
    )
    print("Collection created")
else:
    collection = read_collection(client, cosmos_database, cosmos_collection)
    print("Collection found")
    
dbsecrets = dict(
    Endpoint=endpoint, 
    Masterkey=master_key, 
    Database=cosmos_database, 
    Collection=cosmos_collection, 
    Upsert=True
)

Database found
Collection found


## 2 Training

Next, we train an [Alternating Least Squares model](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html) on [MovieLens](https://grouplens.org/datasets/movielens/) dataset.

### 2.1 Download the MovieLens dataset

In [10]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField(userCol, IntegerType()),
        StructField(itemCol, IntegerType()),
        StructField(ratingCol, FloatType()),
    )
)

data = movielens.load_spark_df(spark, size=MOVIELENS_DATA_SIZE, schema=schema)
data.show()

100%|██████████| 4.81k/4.81k [00:00<00:00, 16.8kKB/s]


+------+-------+------+
|UserId|MovieId|Rating|
+------+-------+------+
|   196|    242|   3.0|
|   186|    302|   3.0|
|    22|    377|   1.0|
|   244|     51|   2.0|
|   166|    346|   1.0|
|   298|    474|   4.0|
|   115|    265|   2.0|
|   253|    465|   5.0|
|   305|    451|   3.0|
|     6|     86|   3.0|
|    62|    257|   2.0|
|   286|   1014|   5.0|
|   200|    222|   5.0|
|   210|     40|   3.0|
|   224|     29|   3.0|
|   303|    785|   3.0|
|   122|    387|   5.0|
|   194|    274|   2.0|
|   291|   1042|   4.0|
|   234|   1184|   2.0|
+------+-------+------+
only showing top 20 rows



### 2.2 Split the data into train, test
There are several ways of splitting the data: random, chronological, stratified, etc., each of which favors a different real-world evaluation use case. We will split randomly in this example – for more details on which splitter to choose, consult [this guide](https://github.com/Microsoft/Recommenders/blob/master/notebooks/01_data/data_split.ipynb).

In [11]:
train, test = spark_random_split(data, ratio=0.75, seed=42)
print("N train", train.cache().count())
print("N test", test.cache().count())

N train 75031
N test 24969


### 2.3 Train the ALS model on the training data

To predict movie ratings, we use the rating data in the training set as users' explicit feedback. The hyperparameters used to estimate the model are set based on [this page](http://mymedialite.net/examples/datasets.html).

Under most circumstances, you would explore the hyperparameters and choose an optimal set based on some criteria. For additional details on this process, please see additional information in the deep dives [here](https://github.com/microsoft/recommenders/blob/master/notebooks/04_model_select_and_optimize/tuning_spark_als.ipynb).

In [12]:
als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=True,
    userCol=userCol,
    itemCol=itemCol,
    ratingCol=ratingCol,
)

In [13]:
model = als.fit(train)

### 2.4 Get top-k recommendations for our testing data

In the movie recommendation use case, recommending movies that have been rated by the users do not make sense. Therefore, the rated movies are removed from the recommended items.

In order to achieve this, we recommend all movies to all users, and then remove the user-movie pairs that exist in the training dataset.

In [14]:
# Get the cross join of all user-item pairs and score them.
users = train.select(userCol).distinct()
items = train.select(itemCol).distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)
dfs_pred.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|   148|    148| 2.9885666|
|   463|    148| 2.4566634|
|   471|    148|  5.211588|
|   496|    148| 2.7297416|
|   833|    148| 1.3390744|
|   243|    148|  2.865699|
|   392|    148| 2.7495515|
|   540|    148| 3.3569064|
|   623|    148| 2.7875235|
|   737|    148|   2.05034|
|   858|    148| 2.3909347|
|   897|    148| 3.0353885|
|    31|    148| 1.9389185|
|   516|    148| 3.8283033|
|    85|    148|  2.354567|
|   137|    148|  4.645254|
|   251|    148| 2.9032419|
|   451|    148| 3.7471225|
|   580|    148| 3.8034196|
|   808|    148| 3.6879075|
+------+-------+----------+
only showing top 20 rows



In [15]:
# Remove seen items.
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train.alias("train"),
    (dfs_pred[userCol]==train[userCol]) & (dfs_pred[itemCol]==train[itemCol]),
    how='outer'
)
top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train."+ratingCol].isNull()) \
    .select("pred."+userCol, "pred."+itemCol, "pred.prediction")

top_all.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|     1|    587| 3.1521533|
|     1|    869|  2.416863|
|     1|   1208| 3.1338184|
|     1|   1677| 3.0138044|
|     2|     80| 2.7144198|
|     2|    303| 4.0214634|
|     2|    472| 2.5440526|
|     2|    582|  4.103753|
|     2|    838| 1.0517997|
|     2|    975|  2.737259|
|     2|   1260| 3.8516903|
|     2|   1325| 1.1345804|
|     2|   1381| 3.3377702|
|     2|   1530| 2.2663925|
|     3|     22| 3.2361977|
|     3|     57|  2.531235|
|     3|     89| 3.7172291|
|     3|    367| 2.3831084|
|     3|   1091| 1.6094522|
|     3|   1167|  3.595107|
+------+-------+----------+
only showing top 20 rows



### 2.5 Evaluate how well ALS performs

Evaluate model performance using metrics such as Precision@K, Recall@K, [MAP@K](https://en.wikipedia.org/wiki/Evaluation_measures_\(information_retrieval\) or [nDCG@K](https://en.wikipedia.org/wiki/Discounted_cumulative_gain). For a full guide on what metrics to evaluate your recommender with, consult [this guide](https://github.com/Microsoft/Recommenders/blob/master/notebooks/03_evaluate/evaluation.ipynb).

In [16]:
cols = {
    'col_user': userCol,
    'col_item': itemCol,
    'col_rating': ratingCol,
    'col_prediction': "prediction",
}

test.show()

+------+-------+------+
|UserId|MovieId|Rating|
+------+-------+------+
|     1|      2|   3.0|
|     1|      3|   4.0|
|     1|      4|   3.0|
|     1|     14|   5.0|
|     1|     17|   3.0|
|     1|     27|   2.0|
|     1|     29|   1.0|
|     1|     35|   1.0|
|     1|     36|   2.0|
|     1|     51|   4.0|
|     1|     52|   4.0|
|     1|     54|   3.0|
|     1|     56|   4.0|
|     1|     60|   5.0|
|     1|     64|   5.0|
|     1|     69|   3.0|
|     1|     77|   4.0|
|     1|     83|   3.0|
|     1|     85|   3.0|
|     1|     88|   4.0|
+------+-------+------+
only showing top 20 rows



In [17]:
# Evaluate Ranking Metrics
rank_eval = SparkRankingEvaluation(
    test, 
    top_all, 
    k=TOP_K,
    **cols
)

print(
    "Model:\tALS",
    "Top K:\t%d" % rank_eval.k,
    "MAP:\t%f" % rank_eval.map_at_k(),
    "NDCG:\t%f" % rank_eval.ndcg_at_k(),
    "Precision@K:\t%f" % rank_eval.precision_at_k(),
    "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n'
)

Model:	ALS
Top K:	10
MAP:	0.002841
NDCG:	0.029330
Precision@K:	0.033298
Recall@K:	0.010961


In [18]:
# Evaluate Rating Metrics
prediction = model.transform(test)
rating_eval = SparkRatingEvaluation(
    test, 
    prediction, 
    **cols
)

print(
    "Model:\tALS rating prediction",
    "RMSE:\t%.2f" % rating_eval.rmse(),
    "MAE:\t%f" % rating_eval.mae(),
    "Explained variance:\t%f" % rating_eval.exp_var(),
    "R squared:\t%f" % rating_eval.rsquared(), sep='\n'
)

Model:	ALS rating prediction
RMSE:	0.95
MAE:	0.743691
Explained variance:	0.284494
R squared:	0.279619


### 2.6 Save the model

In [19]:
(model
 .write()
 .overwrite()
 .save(model_name))

## 3. Operationalize the Recommender Service
Once the model is built with desirable performance, it will be operationalized to run as a REST endpoint to be utilized by a real time service. We will utilize [Azure Cosmos DB](https://azure.microsoft.com/en-us/services/cosmos-db/), [Azure Machine Learning Service](https://azure.microsoft.com/en-us/services/machine-learning-service/), and [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes) to operationalize the recommender service.

### 3.1 Create a look-up for Recommendations in Cosmos DB

First, the Top-10 recommendations for each user as predicted by the model are stored as a lookup table in Cosmos DB. At runtime, the service will return the Top-10 recommendations as precomputed and stored in Cosmos DB:

In [23]:
recs = model.recommendForAllUsers(10)
recs_topk = recs.withColumn("id", recs[userCol].cast("string")) \
    .select("id", "recommendations." + itemCol)
recs_topk.show()

+---+--------------------+
| id|             MovieId|
+---+--------------------+
|471|[1240, 1038, 867,...|
|463|[1195, 958, 1126,...|
|833|[179, 730, 1597, ...|
|496|[320, 1589, 767, ...|
|148|[320, 59, 253, 74...|
|540|[1512, 1642, 1367...|
|392|[1512, 511, 1367,...|
|243|[1512, 1367, 1642...|
|623|[958, 50, 172, 95...|
|737|[1512, 1524, 206,...|
|897|[1368, 320, 1643,...|
|858|[1266, 1195, 1240...|
| 31|[320, 1344, 836, ...|
|516|[1195, 1642, 1005...|
|580|[1512, 1098, 1015...|
|251|[1643, 1154, 127,...|
|451|[1154, 1368, 1218...|
| 85|[1512, 1367, 1643...|
|137|[1154, 169, 1639,...|
|808|[958, 867, 1449, ...|
+---+--------------------+
only showing top 20 rows



In [25]:
# Save data to CosmosDB
(recs_topk.coalesce(1)
 .write
 .format("com.microsoft.azure.cosmosdb.spark")
 .mode('overwrite')
 .options(**dbsecrets)
 .save())

### 3.2 Configure Azure Machine Learning

Next, Azure Machine Learning Service is used to create a model scoring image and deploy it to Azure Kubernetes Service as a scalable containerized service. To achieve this, a **scoring script** and an **environment config** should be created. The following shows the content of the two files.  

In the scoring script, we make a call to Cosmos DB to lookup the top 10 movies to recommend given an input User ID:

In [26]:
score_sparkml = """
import json
import pydocumentdb.document_client as document_client

def init(local=False):
    global client, collection
    try:
        client = document_client.DocumentClient('{endpoint}', dict(masterKey='{key}'))
        collection = client.ReadCollection(collection_link='dbs/{database}/colls/{collection}')
    except Exception as e:
        collection = e

def run(input_json):
    try:
        # Query them in SQL
        id = str(json.loads(json.loads(input_json)[0])['id'])
        query = dict(query='SELECT * FROM c WHERE c.id = "' + id +'"')
        options = dict(partitionKey=str(id))
        document_link = 'dbs/{database}/colls/{collection}/docs/' + id
        result = client.ReadDocument(document_link, options);  
    except Exception as e:
        result = str(e)
    return json.dumps(str(result))
""".format(key=dbsecrets['Masterkey'], 
           endpoint=dbsecrets['Endpoint'], 
           database=dbsecrets['Database'], 
           collection=dbsecrets['Collection'])

# test validity of python string
exec(score_sparkml)

with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

Register your model:

In [27]:
mymodel = Model.register(
    model_path=model_name,  # this points to a local file
    model_name=model_name,  # this is the name the model is registered as
    description="AML trained model",
    workspace=ws
)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model mvl-als-reco.mml
mvl-als-reco.mml AML trained model 1


### 3.3 Deploy the model as a Service on AKS

#### 3.3.1 Create an Environment for your model:

In [28]:
env = Environment(name='sparkmlenv')

# Specify a public image from microsoft/mmlspark as base image
env.docker.base_image="microsoft/mmlspark:0.15"

pip = [
    'azureml-defaults', 
    'numpy==1.14.2', 
    'scikit-learn==0.19.1', 
    'pandas', 
    'pydocumentdb'
]

# Add dependencies needed for inferencing
env.python.conda_dependencies = CondaDependencies.create(pip_packages=pip)
env.inferencing_stack_version = "latest"

# Add spark packages
env.spark.precache_packages = True
env.spark.repositories = ["https://mmlspark.azureedge.net/maven"]
env.spark.packages= [
    SparkPackage("com.microsoft.ml.spark", "mmlspark_2.11", "0.15"),
    SparkPackage("com.microsoft.azure", artifact="azure-storage", version="2.0.0"),
    SparkPackage(group="org.apache.hadoop", artifact="hadoop-azure", version="2.7.0")
]

#### 3.3.2 Create an AKS Cluster to run your container
This may take 20 to 30 minutes depending on the cluster size.

In [29]:
# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    # Create the cluster using the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()
    aks_target = ComputeTarget.create(
        workspace=ws, 
        name=aks_name, 
        provisioning_configuration=prov_config
    )
    aks_target.wait_for_completion(show_output = True)
    print(aks_target.provisioning_state)
    # To check any error logs, print(aks_target.provisioning_errors)

Creating..............................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded


#### 3.3.3 Deploy the container image to AKS:

In [30]:
# Create an Inferencing Configuration with your environment and scoring script
inference_config = InferenceConfig(
    environment=env,
    entry_script="score_sparkml.py"
)

# Set the web service configuration (using default here with app insights)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True)

# Webservice creation using single command
try:
    aks_service = Model.deploy(
        workspace=ws,
        models=[mymodel],
        name=service_name,
        inference_config=inference_config,
        deployment_config=aks_config,
        deployment_target=aks_target
    )
    aks_service.wait_for_deployment(show_output=True)
except WebserviceException:
    # Retrieve existing service.
    aks_service = Webservice(ws, name=service_name)
    print("Retrieved existing service")

Running........................................................................................................................................
SucceededAKS service creation operation finished, operation "Succeeded"


### 3.4 Call the AKS model service
After the deployment, the service can be called with a user ID – the service will then look up the top 10 recommendations for that user in Cosmos DB and send back the results.
The following script demonstrates how to call the recommendation service API and view the result for the given user ID:

In [159]:
import json

scoring_url = aks_service.scoring_uri
service_key = aks_service.get_keys()[0]

input_data = '["{\\"id\\":\\"496\\"}"]'.encode()

req = urllib.request.Request(scoring_url,data=input_data)
req.add_header("Authorization","Bearer {}".format(service_key))
req.add_header("Content-Type","application/json")

with Timer() as t: 
    with urllib.request.urlopen(req) as result:
        res = result.read()
        resj = json.loads(
            # Cleanup to parse into a json object
            res.decode("utf-8")
            .replace("\\", "")
            .replace('"', "")
            .replace("'", '"')
        )
        print(json.dumps(resj, indent=4))
    
print("Full run took %.2f seconds" % t.interval)

{
    "MovieId": [
        320,
        1589,
        767,
        1104,
        1167,
        1137,
        919,
        1344,
        1400,
        61
    ],
    "id": "496",
    "_rid": "BbQDALSshkBiAQAAAAAACA==",
    "_self": "dbs/BbQDAA==/colls/BbQDALSshkA=/docs/BbQDALSshkBiAQAAAAAACA==/",
    "_etag": "0400d3e4-0000-0100-0000-5dec0dc70000",
    "_attachments": "attachments/",
    "_ts": 1575751111
}
Full run took 0.13 seconds
